In [7]:
import numpy as np
import pandas as pd
import pickle
import math
import re
from matplotlib import pyplot as P
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.utils.vis_utils import plot_model
from keras.callbacks import TensorBoard
from keras import backend as K

from time import time
from infodenguepredict.data.infodengue import combined_data, get_alerta_table, get_city_names, build_multicity_dataset
from infodenguepredict.models.deeplearning.preprocessing import split_data, normalize_data
from infodenguepredict.predict_settings import *


In [8]:
from infodenguepredict.models.deeplearning.lstm import *
from infodenguepredict.predict_settings import *

In [9]:
import itertools
import matplotlib.pyplot as plt

In [10]:
from sklearn import cross_validation
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import f1_score

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
def class_report(y_true, y_predict, title='Avaliação', target_names=['Classe 0', 'Classe 1',]):
    print('Classification Report for feature: {}'.format(title))
    print(classification_report(y_true, y_predict, target_names=target_names))
    print('ROC Score: {}'.format(roc_auc_score(y_true, y_predict)))
    print('Accuracy Score: {}'.format(accuracy_score(y_true, y_predict)))
    print('Average Precision Score: {}'.format(average_precision_score(y_true, y_predict)))
    print('f1 Score: {}'.format(f1_score(y_true, y_predict)))

/home/elisa/.local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [11]:
def single_prediction(city, state, predictors, predict_n, look_back, hidden, epochs, predict=False):
    """
    Fit an LSTM model to generate predictions for a city, Using its cluster as regressors.
    :param city: geocode of the target city
    :param state: State containing the city
    :param predict_n: How many weeks ahead to predict
    :param look_back: Look-back time window length used by the model
    :param hidden: Number of hidden layers in each LSTM unit
    :param epochs: Number of epochs of training
    :param random: If the model should be trained on a random selection of ten cities of the same state.
    :return:
    """

    with open('infodenguepredict/analysis/clusters_{}.pkl'.format(state), 'rb') as fp:
        clusters = pickle.load(fp)
    data, group = get_cluster_data(geocode=city, clusters=clusters,
                                   data_types=DATA_TYPES, cols=predictors)

    indice = list(data.index)
    indice = [i.date() for i in indice]

    city_name = get_city_names([city, 0])[0][1]
    if predict:
        ratio = 1
    else:
        ratio = 0.7

    predicted, X_test, Y_test, Y_train, factor = train_evaluate_model(city, data, predict_n, look_back,
                                                              hidden, epochs, ratio=ratio)
    plot_predicted_vs_data(predicted,
                           np.concatenate((Y_train, Y_test), axis=0),
                           indice[:],
                           label='Predictions for {}'.format(city_name),
                           pred_window=predict_n,
                           factor= factor,
                           split_point=len(Y_train))

    return predicted, X_test, Y_test, factor


In [25]:
def create_Q(city, state):
    data = get_alerta_table(CITY, 'RJ')
    data = data[['casos', 'SE']].reset_index(drop=True)
    
    data.SE = [str(i)[-2:] for i in data.SE]
    Q = data.groupby('SE').median()
    Q['std'] = data.groupby('SE').std()['casos']
    return Q, data.SE.values

def build_y_test(row,k):
    if abs(row['real'] - row['mediana']) < k*row['std']:
        return 1
    else:
        return 0
    
def build_y_pred(row, k):
    if abs(row['preds'] - row['mediana']) < k*row['std']:
        return 1
    else:
        return 0
    
def matrix(city, preds, real, Q, SE, k):
    df = pd.DataFrame({'preds':preds, 'SE':SE[-len(preds):], 'real':real})
    to_merge = pd.DataFrame({'mediana':Q['casos'.format(city)],
                            'std':Q['std']}).reset_index()    
    df = df.merge(to_merge,how='left', on='SE')  
    
    df['y_test'] = df.apply(build_y_test, axis=1, args=(k,))
    df['y_pred'] = df.apply(build_y_pred, axis=1, args=(k,))
    cnf_matrix = confusion_matrix(df.y_test, df.y_pred)
    class_report(df.y_test, df.y_pred)
    plot_confusion_matrix(cnf_matrix, classes=('not_expected','expected'))
    

In [22]:
len(Y_test)

123

In [14]:
predict_n=PREDICTION_WINDOW
look_back=LOOK_BACK
hidden=HIDDEN
epochs=EPOCHS

In [27]:
predicted, X_test, Y_test, factor = single_prediction(CITY, STATE, PREDICTORS, predict_n=4, look_back=4, 
                  hidden=HIDDEN, epochs=100)


/home/elisa/.local/lib/python3.6/site-packages/pandas/core/groupby.py:842: RuntimeWarning: Mean of empty slice
  f = lambda x: func(x, *args, **kwargs)


OperationalError: (psycopg2.OperationalError) server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.
 [SQL: 'select geocodigo, nome from "Dengue_global"."Municipio" WHERE geocodigo in (3304557, 0);'] (Background on this error at: http://sqlalche.me/e/e3q8)

In [ ]:
preds = predicted[:,3]*factor
preds = preds[-len(Y_test):]

real = Y_test[:,3]*factor

Q, SE = create_Q(CITY, 'RJ')

In [ ]:
Y_test

In [ ]:
matrix(CITY, preds,real, Q, SE, 0.4)

In [141]:
from sklearn.metrics import confusion_matrix

In [15]:
# def rank_cities(state):
#     mult = build_multicity_dataset(state)
#     cols = list(filter(re.compile('casos_\d+').search, mult.columns))
#     mult = mult[cols]

#     print(mult.head())
# #     codes = pd.read_excel('../../data/codigos_{}.xlsx'.format(state),
# #                           names=['city', 'code'], header=None).set_index('code').T

#     ints = pd.DataFrame()
#     for col in mult.columns:
#         # ints.loc[codes[int(re.sub('casos_', '', col))]] = [np.trapz(mult[col])]
#         ints[col] = [np.trapz(mult[col])]
#     return ints

In [17]:
# rank = rank_cities('RJ')

            casos_3300100  casos_3300159  casos_3300209  casos_3300225  \
data_iniSE                                                               
2010-01-03              9              0              1              0   
2010-01-10              5              0              0              0   
2010-01-17              4              0              0              0   
2010-01-24             11              0              0              0   
2010-01-31              6              0              5              0   

            casos_3300233  casos_3300258  casos_3300308  casos_3300407  \
data_iniSE                                                               
2010-01-03              0              0              0              0   
2010-01-10              0              1              0              3   
2010-01-17              0              0              2              4   
2010-01-24              0              0              1              0   
2010-01-31              0            

In [ ]:
# for col in rank:
#     city = re.sub('casos_', '', col)
#     metric = single_prediction(int(city), state, PREDICTORS, predict_n=PREDICTION_WINDOW, look_back=LOOK_BACK,
#                       hidden=HIDDEN, epochs=EPOCHS)
#     mapes.append(metric)

/home/elisa/.local/lib/python3.6/site-packages/pandas/core/groupby.py:842: RuntimeWarning: Mean of empty slice
  f = lambda x: func(x, *args, **kwargs)


283
(279, 4, 90) (279, 5) (118, 4, 90) (118, 5)
Compilation Time :  0.034201622009277344
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (1, 4, 4)                 1520      
_________________________________________________________________
lstm_5 (LSTM)                (1, 4, 4)                 144       
_________________________________________________________________
lstm_6 (LSTM)                (1, 4)                    144       
_________________________________________________________________
dense_2 (Dense)              (1, 5)                    25        
Total params: 1,833
Trainable params: 1,833
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.6/dist-packages/keras/models.py:874: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 237 samples, validate on 42 samples
Epoch 1/50
237/237 [==============================] - 5s 19ms/step - loss: 0.0149 - acc: 0.2405 - mean_absolute_percentage_error: 1462398.7292 - val_loss: 0.0644 - val_acc: 0.2857 - val_mean_absolute_percentage_error: 2941026.9130
Epoch 2/50
237/237 [==============================] - 4s 17ms/step - loss: 0.0143 - acc: 0.2278 - mean_absolute_percentage_error: 1764576.4369 - val_loss: 0.0640 - val_acc: 0.3095 - val_mean_absolute_percentage_error: 3040827.0359
Epoch 3/50
237/237 [==============================] - 5s 21ms/step - loss: 0.0134 - acc: 0.2405 - mean_absolute_percentage_error: 1473952.6556 - val_loss: 0.0526 - val_acc: 0.3810 - val_mean_absolute_percentage_error: 2794852.8707
Epoch 4/50
237/237 [==============================] - 8s 32ms/step - loss: 0.0135 - acc: 0.2658 - mean_absolute_percentage_error: 1433507.7401 - val_loss: 0.0707 - val_acc: 0.2619 - val_mean_absolute_percentage_error: 773158.6508
Epoch 5/50
237/237 [============

/home/elisa/.local/lib/python3.6/site-packages/pandas/core/groupby.py:842: RuntimeWarning: Mean of empty slice
  f = lambda x: func(x, *args, **kwargs)


...

Epoch 36/50
237/237 [==============================] - 5s 20ms/step - loss: 0.0034 - acc: 0.5443 - mean_absolute_percentage_error: 3822494.3443 - val_loss: 0.0012 - val_acc: 0.5714 - val_mean_absolute_percentage_error: 9909629.0595
Epoch 37/50
237/237 [==============================] - 5s 20ms/step - loss: 0.0036 - acc: 0.5738 - mean_absolute_percentage_error: 2331014.5708 - val_loss: 0.0015 - val_acc: 0.5952 - val_mean_absolute_percentage_error: 12073870.4158
Epoch 38/50
237/237 [==============================] - 5s 22ms/step - loss: 0.0032 - acc: 0.5612 - mean_absolute_percentage_error: 1598688.4729 - val_loss: 0.0031 - val_acc: 0.5952 - val_mean_absolute_percentage_error: 19939362.2262
Epoch 39/50
237/237 [==============================] - 5s 21ms/step - loss: 0.0032 - acc: 0.5781 - mean_absolute_percentage_error: 1823546.7933 - val_loss: 0.0033 - val_acc: 0.5238 - val_mean_absolute_percentage_error: 21325287.6484
Epoch 40/50
237/237 [==============================] - 6s 23ms/step -

/home/elisa/.local/lib/python3.6/site-packages/pandas/core/groupby.py:842: RuntimeWarning: Mean of empty slice
  f = lambda x: func(x, *args, **kwargs)


283
(279, 4, 90) (279, 5) (118, 4, 90) (118, 5)
Compilation Time :  0.032843589782714844
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_58 (LSTM)               (1, 4, 4)                 1520      
_________________________________________________________________
lstm_59 (LSTM)               (1, 4, 4)                 144       
_________________________________________________________________
lstm_60 (LSTM)               (1, 4)                    144       
_________________________________________________________________
dense_20 (Dense)             (1, 5)                    25        
Total params: 1,833
Trainable params: 1,833
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.6/dist-packages/keras/models.py:874: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 237 samples, validate on 42 samples


In [ ]:
# rank = rank.T
# rank['mape'] = mapes
# rank.to_pickle('rank.pkl')